# Продвинутый Python, лекция 6

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## Базы данных

![](https://habrastorage.org/r/w1560/getpro/habr/upload_files/1e6/32d/9c0/1e632d9c081a0ab5591f11a3764571b3.jpg)

Вначале немного подушним (но полезно подушним). Есть два понятия: БД (база данных) и СУБД (система управления базы данных). В чем разница?

* БД - набор данных (таблиц), логически связанных между собой, буквально место, где они находятся

* СУБД - набор инструментов, которые позволяют что-то делать с БД (осуществляет connection к БД для операций)

Очевидно, что устройство БД и СУБД связаны друг с другом (достаточно странно, напрмиер, есть суп вилкой или лапшу ложкой)

Какие СУБД бывают? Реляционные и нереляционные

* Реляционные - нужна структура, нормальные формы etc (MySQL, PostgreSQL)

* Нереляционные - можно хранить что угодно и примерно как угодно (MongoDB, концепция DataLake)

Мы будем говорить про реляционные (потому что это стандарт, SQL, но различаются между СУБД), на следующей лекции поговорим про нереляционные (так называемые NoSQL и иже с ними)

## Интро в SQL

Чтобы понимать, как работать с СУБД, разберем, на каком языке говорят все реляционные СУБД - а говорят они не на Питоне, а на SQL.

SQL (Structured Query Language) - язык запросов для обращения и выполнения операций реляционной БД. Скорее всего вы слышали что-нибудь из списка: MySQL, PostgreSQL, Oracle SQL, KQL и так далее. Это все различные диалекты, у которых есть общая база и дальше разные доработки/изменения, что-то может по-другому называться, но суть одинаковая. Вот про эту суть и поговорим

Что из себя представляет таблица? Набор строк и столбцов, где любые операции начинаются с того, что вы выбираете нужный срез. На самом деле, достаточно просто понять SQL, если вспомните, как работает pandas.

Базовый запрос к таблице:

```
SELECT * -- выбери все
FROM table -- из данной таблицы
```

В качестве выбора в селекте мы указываем столбцы:

```
SELECT DISTINCT -- уникальные значения
    column1 as c, -- колонки можно переименовывать, а еще можно создавать
    column2,
    column1 + column2 as s,
    ...
FROM table
LIMIT 15 -- выведи только первые 15 значений
```

Как выбрать только нужные строки? С помощью ограничений по столбцам:

```
SELECT
    column1,
    column2,
    ...
FROM table
WHERE column1 != <value>
AND column2 IN (<values>)
OR column3 IS NOT NULL
AND column4 + column5 >= <value>
```

Что еще умеем? Группировки!

```
SELECT
    SUM/AVG/MIN/MAX/MEDIAN/AGGREGATE(column1) as c -- функции аггрегации
    column2,
    column3
FROM table
GROUP BY column2, column3 -- по чему группируем (ВАЖНО: тут будут минимум все неагрегированные столбцы из запроса)
```

Отдельно умеем фильтровать по агрегациям:

```
SELECT
    SUM/AVG/MIN/MAX/MEDIAN/AGGREGATE(column1) as c -- функции аггрегации
    column2,
    column3
FROM table
GROUP BY column2, column3 -- по чему группируем (ВАЖНО: тут будут минимум все неагрегированные столбцы из запроса)
HAVING c > <value>
```

И последнее, что базово надо знать: подзапросы и JOINы

```
SELECT *
FROM table
WHERE column1 IN (SELECT col FROM table_2)
```

```
SELECT
    t.*,
    t2.*
FROM table as t
LEFT/RIGHT/INNER/OUTER JOIN table_2 as t2
ON t.column = t2.col
```




![](https://user-images.githubusercontent.com/19540357/65278347-7b417c80-db1b-11e9-825d-280605deaccc.png)

Общая картинка (что и в каком порядке):

```
SELECT ...
FROM table
JOIN table_2 ON ...
WHERE ...
GROUP BY ...
HAVING ...
LIMIT ...
```

## Операции и иже с ним

Любая операция по изменению данных должна соблюдать ACID-свойства. Это свойства, необходимые для поддержания любой системы в порядке. Что это такое?

* A - Atomicity (атомарность)

Любая транзакция должны быть выполнена полностью или не быть выполнена совсем. Отличный пример с переводом денег: деньги надо снять с одного счета и положить на другой. Не должно быть такой ситуации, что выполнилась только 1 часть, должны выполниться либо обе части, либо ни одна.

* C - Consistency (консистентность)

Любая транзакция должна сохранять консистентность данных. Пример: у нас есть таблица с телефонами (по типу UserId - PhoneNumber) и таблица с пользователем (UserId - FirstName - SecondName), и при регистрации надо добавить запись и в пользователей, и в телефоны, иначе теряется консистентность

* I - Isolation (изоляция)

Вспоминаем threads и race condition. За счет чего появляется проблема и как она чинится? Такую же проблему нужно учитывать и при транзакциях в БД

* D - Durability (надежность)

Любая выполненная транзакция выполнена, вне зависимости от других условий (нельзя говорить, что операция выполнена до тех пор, пока мы ее не выполнили)

## Нормальные формы

Как правильно проектировать собственные БД? Если мы говорим про реляционные БД, то на это есть целый ответ: проводить нормализацию (или приводить к так называемым нормальным формам)

Что это такое и зачем это нужно? Начнем с последнего:

* Избегаем избыточности (не храним одно и то же несколько раз)

* Избегаем всевозможных аномалий данных при обновлении

* Упрощаем жизнь

То есть логика понятная - пытаемся выстроить систему, где все лежит на своем месте

Что такое нормальные формы? Условия, которые БД должна содержать для достижения данного состояния. Выделяют 6 нормальных форм (одна сложнее другой), в идеале все смотрят на 3НФ (третья нормальная форма)

* 1НФ - есть маппинг 1:1 в значениях строк (одна запись - одна строка) и данные в каждом столбце про одно и то же (один домен)

* 2НФ - 1НФ + каждый неключевые значения непосредственно зависят от всего ключа

* 3НФ - 2НФ + каждый атрибут должен предоставлять информацию о ключе, полном ключе и ни о чём, кроме ключа

Давайте на примере:

У нас изначально есть вот такая табличка:

\begin{array}{cccс}
\text{Фамилия}&\text{Имя}&\text{Должность}&\text{Персональная информация}\\
Петров&Тимур&Попугай&79000000000; example1@mail.com \\
Дешеулин&Олег&Олег&example2@mail.com \\
Бузаев&Федор&Daddy&79000000002
\end{array}

Она не в 1НФ за счет того, что находится в Персональной информации, давайте исправим:

\begin{array}{cccс}
\text{Фамилия}&\text{Имя}&\text{Должность}&\text{Телефон}&\text{Почта}\\
Петров&Тимур&Попугай&79000000000&example1@mail.com \\
Дешеулин&Олег&Олег&NULL&example2@mail.com \\
Бузаев&Федор&Daddy&79000000002&NULL
\end{array}

Допустим, что у нас добавилась зарплата, которая зависит от должности:

\begin{array}{cccс}
\text{Фамилия}&\text{Имя}&\text{Должность}&\text{Телефон}&\text{Почта}&\text{ЗП}\\
Петров&Тимур&Попугай&79000000000&example1@mail.com&0 \\
Дешеулин&Олег&Олег&NULL&example2@mail.com&300000 \\
Бузаев&Федор&Daddy&79000000002&NULL&150000
\end{array}

Таблица перестала быть в 2НФ (потому что связь не с ФИО + должность, а только от должности). Тогда должно быть 2 таблицы:

Таблица 1:

\begin{array}{cccс}
\text{Фамилия}&\text{Имя}&\text{Должность}&\text{Телефон}&\text{Почта}\\
Петров&Тимур&Попугай&79000000000&example1@mail.com \\
Дешеулин&Олег&Олег&NULL&example2@mail.com \\
Бузаев&Федор&Daddy&79000000002&NULL
\end{array}

Таблица 2:


\begin{array}{cccс}
\text{Должность}&\text{ЗП}\\
Попугай&0 \\
Олег&300000 \\
Daddy&150000
\end{array}

А теперь еще и добавим оператора связи:

Таблица 1:

\begin{array}{cccс}
\text{Фамилия}&\text{Имя}&\text{Должность}&\text{Телефон}&\text{Почта}&\text{Оператор связи}\\
Петров&Тимур&Попугай&79000000000&example1@mail.com&A \\
Дешеулин&Олег&Олег&NULL&example2@mail.com&NULL \\
Бузаев&Федор&Daddy&79000000002&NULL&B
\end{array}

Таблица 2:


\begin{array}{cccс}
\text{Должность}&\text{ЗП}\\
Попугай&0 \\
Олег&300000 \\
Daddy&150000
\end{array}

Формально все данные все еще в 2НФ: от связки Имя + Фамилия + Должность зависит оператор связи. Но тем не менее, есть связь Телефон -> Оператор связи, от которой мы как раз и избавляемся (по сути 3НФ ровно про это: никаких транзитивных штук):

Таблица 1:

\begin{array}{cccс}
\text{Фамилия}&\text{Имя}&\text{Должность}&\text{Телефон}&\text{Почта}\\
Петров&Тимур&Попугай&79000000000&example1@mail.com \\
Дешеулин&Олег&Олег&NULL&example2@mail.com \\
Бузаев&Федор&Daddy&79000000002&NULL
\end{array}

Таблица 2:

\begin{array}{cccс}
\text{Должность}&\text{ЗП}\\
Попугай&0 \\
Олег&300000 \\
Daddy&150000
\end{array}

Таблица 3:

\begin{array}{cccс}
\text{Телефон}&\text{Оператор}\\
79000000000&A \\
79000000002&B \\
\end{array}

Все, поговорили про все, прежде чем тыкаться в Python. Давайте теперь к нему!

## Как подключаться к БД?

Первое, что нам нужно при любой работе - это уметь подключаться к базе данных. А как, а что?

База данных обычно лежит на отдельном сервере, где и крутится сама по себе (что-то обновляет etc). А приложение на другом сервере к нему подключается. Поэтому надо уметь подключаться :з

Подключение идет через т.н. драйвера, но люди уже давно этим занимаются, поэтому подключение происходит через менеджер драйверов, который позволяет легко подключиться практически к любой СУБД:

* ODBC (Open Database Connectivity)

```
Driver=(SQL Server);Server=(serverName);Database=(instanceName)(;property=value)
```

* JDBC (Java Database Connectivity)

```
Driver=(SQL Server);Server=(serverName);Database=(instanceName)(;property=value)
```

* Кастомные драйвера

В общем виде что должно быть при подключении:

- тип СУБД

- хост

- порт

- название БД

- данные для авторизации (логин-пароль)

## Теперь к Python

In [ ]:
!pip install pyodbc #ODBC
!pip install jaydebeapi #JDBC
!pip install psycopg2 #PostgreSQL (отдельно, не мучаясь с менеджерами)

Все коннекторы к БД внутри Python должны подчиняться [pep249](https://peps.python.org/pep-0249/) - спецификация для бибилотек с БД. Как выглядит абсолютно любая библиотека:

![](https://i.pinimg.com/originals/be/c7/c1/bec7c10b61f5cb2b851d04f0fbeeee2e.png)

Метод connect - возращает объект типа Connection (связь с БД непосредственно), который должен обладать методами:

* cursor() - получить объект типа cursor (для доступа к данным)

* commit(), rollback() - необходимо для транзакций

* close() - закрыть связь (всегда надо, потому что держит ресурсы)

Разберем это все на базовом СУБД - sqlite3 (есть всегда)

In [ ]:
!wget https://github.com/Palladain/Deep_Python_2023/raw/main/week07/chinook.db

In [2]:
import sqlite3

connection = sqlite3.connect("chinook.db") # открыли
connection.close() # закрыли

![](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

Что такое cursor?

Cursor - это объект для выполнения запросов и получения результатов (по сути объект контеста для запроса, а мы где вообще)

И cursor тоже стоит закрывать (но вообще он закрывается вместе с connection) :)

In [ ]:
connection = sqlite3.connect("chinook.db") # открыли
cursor = connection.cursor()
set(dir(cursor)) - set(dir(object)) #смотрим на методы
# execute - выполни запрос
# fetch - дай результаты

{'__iter__',
 '__next__',
 'arraysize',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'executescript',
 'fetchall',
 'fetchmany',
 'fetchone',
 'lastrowid',
 'row_factory',
 'rowcount',
 'setinputsizes',
 'setoutputsize'}

In [ ]:
cursor.execute("SELECT name, type, rootpage FROM sqlite_master WHERE type='table'")

In [ ]:
one_row = cursor.fetchone() # дай одну строку, 1 строка - 1 кортеж
some_rows = cursor.fetchmany(5) # дай много строк
remain_rows = cursor.fetchall() # дай все остальные строки
print(one_row)
print('-' * 30)
print(some_rows)
print('-' * 30)
print(remain_rows)
print('-' * 30)
# Курсор буквально ходит по таблице
print(cursor.fetchall())

('albums', 'table', 2)
------------------------------
[('sqlite_sequence', 'table', 3), ('artists', 'table', 4), ('customers', 'table', 5), ('employees', 'table', 8), ('genres', 'table', 10)]
------------------------------
[('invoices', 'table', 11), ('invoice_items', 'table', 13), ('media_types', 'table', 15), ('playlists', 'table', 16), ('playlist_track', 'table', 17), ('tracks', 'table', 20), ('sqlite_stat1', 'table', 864)]
------------------------------
[]


## Что умеем делать?

Надобно делать запросы, и получать ответ. А как писать запросы? Есть несколько вариантов:

* Чисто с помощью SQL - кайф, но не очень удобно (мы же в Питоне)

* Query Builder - удобнее, но нужна отдельная библиотека

* ORM - уровень посложнее

### Часть 1. Сырые запросы

In [ ]:
def search_by_name(name, cursor):
    cursor.execute(f"SELECT * FROM playlists WHERE Name LIKE '%{name}%'")
    return cursor.fetchall()

print(search_by_name("Brazil", cursor))

[(11, 'Brazilian Music')]


Проблема - легко ломается! (можно передать какую-нибудь дичь, SQL injection etc). Давайте делать все более безопасно - через словари:

In [ ]:
cursor.execute("SELECT * FROM playlists WHERE Name LIKE :name", {"name": f'%{name}%'})
# двоеточие - название переменных из словаря, драйвер сам все сделает, а также делает экранирование (и преобразование типов)

Но дополнительная сила делать все через словари - это возможность сделать сразу несколько запросов в одном! (например, для вставки значений)

In [ ]:
cursor.executemany("""
INSERT INTO tracks (Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice)
VALUES(?, ?, ?, ?, ?, ?, ?, ?)
""", [
    ("name", 1, 1, 1, None, 1, 0, 0),
    ("name_2", 2, 1, 1, None, 1, 0, 0)
])
cursor.execute("SELECT * FROM tracks WHERE Name LIKE 'name%'")
cursor.fetchall()

[(3504, 'name', 1, 1, 1, None, 1, 0, 0),
 (3505, 'name_2', 2, 1, 1, None, 1, 0, 0)]

### Транзакции

Посмотрим на вот этот код:

In [ ]:
connection_2 = sqlite3.connect("chinook.db") # открыли
cursor_2 = connection_2.cursor()
cursor_2.execute("SELECT * FROM tracks WHERE Name LIKE 'name%'")
cursor_2.fetchall()

[]

Вообще ничего нет. А где...

Если посмотреть в файлы ноутбука, то мы увидим такую вещь, как chinook.db-journal. Это история всех изменений. Мы их сделали, но не закоммитили! А чтобы это сделать, надо сделать commit()

In [ ]:
connection.commit() # сделать commit
# connection.rollback() # отменить все это добро, до commit() надо делать, так как после commit не откатить

In [ ]:
cursor_2.execute("SELECT * FROM tracks WHERE Name LIKE 'name%'")
cursor_2.fetchall()

[(3504, 'name', 1, 1, 1, None, 1, 0, 0),
 (3505, 'name_2', 2, 1, 1, None, 1, 0, 0)]

In [ ]:
connection.close()
connection_2.close()

### Часть 2. Query Builder

В чем плюсы?

1. Вы не пишите SQL код

2. Делаете по частям (логическим), форматировать не надо

3. Можно с разными диалектами (иногда может в ногу выстрелить)

4. Чуть менее гибкий (но редко когда на это наткнешься, не видел я таких)

Но запрос руками, обработка руками (все равно ручками много что делать). Это все библиотека [PyPika](https://pypika.readthedocs.io/en/latest/) (а есть еще другие, но по сути, это одно и то же, вообще в теории и руками можно написать)

In [ ]:
!pip install pypika

#### Как формируются запросы?

In [ ]:
from pypika import Query

q = Query.from_('tracks').select("TrackId", "Name").limit(10) #создаем запрос, после from_ можно вообще в любом порядке писать
q_1 = Query.from_('tracks').limit(10).select("TrackId", "Name")
str(q), str(q_1) # Это все просто текст, запроса к БД еще нет

('SELECT "TrackId","Name" FROM "tracks" LIMIT 10',
 'SELECT "TrackId","Name" FROM "tracks" LIMIT 10')

#### Подбор под диалекты:

In [ ]:
from pypika import MSSQLQuery, OracleQuery, PostgreSQLQuery

oracle = OracleQuery.from_('tracks').select("TrackId", "Name").limit(10)
ms = MSSQLQuery.from_('tracks').select("TrackId", "Name").limit(10) # в MSSQL не существует LIMIT))0)
postgre = PostgreSQLQuery.from_('tracks').select("TrackId", "Name").limit(10)
str(oracle), str(ms), str(postgre)

('SELECT TrackId,Name FROM tracks LIMIT 10',
 'SELECT "TrackId","Name" FROM "tracks" OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY',
 'SELECT "TrackId","Name" FROM "tracks" LIMIT 10')

#### Давайте усложнять и упрощать!

In [ ]:
from pypika import Table, Field # более явный способ разметки

tracks = Table("tracks")
q = Query.from_(tracks) \
    .where(Field('Name').like('%Music%')).select("TrackId", 'Name') \
    .where(Field('TrackId') < 100)

print(str(q))

q = Query.from_(tracks) \
    .where(Field('Name').like('\' OR 1=1')).where(tracks.TrackId < 100) \
    .select('TrackId', 'Name') #прикол экранирования и SQL-injection

print(str(q))

SELECT "TrackId","Name" FROM "tracks" WHERE "Name" LIKE '%Music%' AND "TrackId"<100
SELECT "TrackId","Name" FROM "tracks" WHERE "Name" LIKE ''' OR 1=1' AND "TrackId"<100


#### Ну и параметры:

In [ ]:
from pypika import Parameter

q = Query.from_(tracks) \
    .where(Field('Name').like(Parameter(':name'))).where(tracks.TrackId < 100) \
    .select('TrackId', 'Name') #если не делать параметр, то он посчитает это просто строкой

print(str(q))

SELECT "TrackId","Name" FROM "tracks" WHERE "Name" LIKE :name AND "TrackId"<100


#### Далее у нас наши любимые JOINы:

In [ ]:
albums = Table('albums')

q = q.inner_join(albums)
q = q.using('AlbumId') #второй вариант, суть такая же
#q = q.on(tracks.AlbumId == albums.AlbumId) # Первый вариант
q = q.select(albums.Title)
str(q)

'SELECT "tracks"."TrackId","tracks"."Name","albums"."Title","albums"."Title" FROM "tracks" JOIN "albums" ON "tracks"."AlbumId"="albums"."AlbumId" JOIN "albums" USING ("AlbumId") WHERE "Name" LIKE :name AND "tracks"."TrackId"<100'

#### Подзапросы:

In [ ]:
import pypika.functions as fn
from pypika import Order

q1 = Query.from_(tracks).groupby('GenreId').select('GenreId', fn.Count('*').as_('num_tracks'))
print(str(q1))

num_tracks = Field('num_tracks')
q2 = Query.from_('genres').join(q1).using('GenreId').select('Name', num_tracks).orderby(num_tracks, order=Order.desc).limit(10)

print(str(q2))

SELECT "GenreId",COUNT(*) "num_tracks" FROM "tracks" GROUP BY "GenreId"
SELECT "genres"."Name","num_tracks" FROM "genres" JOIN (SELECT "GenreId",COUNT(*) "num_tracks" FROM "tracks" GROUP BY "GenreId") "sq0" USING ("GenreId") ORDER BY "num_tracks" DESC LIMIT 10


#### А теперь в Sqlite3

In [ ]:
connection = sqlite3.connect('chinook.db')

def get_res(query):
    cursor = connection.cursor()
    cursor.execute(str(query))
    cols = [full_column_info[0] for full_column_info in cursor.description]
    res = cursor.fetchall()
    return [{name: value for name, value in zip(cols, r)} for r in res]

get_res(q2)

[{'Name': 'Rock', 'num_tracks': 1299},
 {'Name': 'Latin', 'num_tracks': 579},
 {'Name': 'Metal', 'num_tracks': 374},
 {'Name': 'Alternative & Punk', 'num_tracks': 332},
 {'Name': 'Jazz', 'num_tracks': 130},
 {'Name': 'TV Shows', 'num_tracks': 93},
 {'Name': 'Blues', 'num_tracks': 81},
 {'Name': 'Classical', 'num_tracks': 74},
 {'Name': 'Drama', 'num_tracks': 64},
 {'Name': 'R&B/Soul', 'num_tracks': 61}]

### Часть 3. ORM

Ну и теперь вершина - ORM (Object-Relational Mapping)

Плюсы:

* Еще удобнее, чем Query Builder (вообще SQL не почувствуем)

* Таблицы - почти DataClass

* Результат автоматически отображается в красивые объекты

Минусы:

* Иногда нафиг не надо (бывает хуже производительность, на простых примерах из 1-2 таблицы не надо)

Пример: [SQLAlchemy](https://www.sqlalchemy.org/)

Выглядит это вот так:

![](https://s3.amazonaws.com/media-p.slid.es/uploads/10882/images/5116660/sqlalchemy__3_.png)

* DB API - все, что про cursor, connection etc

* Core - это похожее на Query Builder структура, возможность создавать запросы

* ORM - уже модуль для маппинга результатов к объектам

#### Движок

In [3]:
!pip install sqlalchemy

In [4]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

# Единственное место с подсоединением к СУБД
engine = create_engine('sqlite+pysqlite:///chinook.db', echo=True)
Base = declarative_base() # предок для всех моделей (таблиц)

<ipython-input-4-22dd3c689ed3>:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base() # предок для всех моделей (таблиц)


#### Модели

In [5]:
class Tracks(Base):
    __tablename__ = 'tracks' # имя таблицы

    track_id = Column(Integer, name='TrackId', primary_key=True) # обязательно должен быть PrimaryKey
    Name = Column(String)
    GenreId = Column(Integer)
    UnitPrice = Column(Float)
    MediaTypeId = Column(Integer)
    Milliseconds = Column(Integer)

    # создаем колонки, дублируя БД

#### Сессия

Что такое Session? По сути дела Connection в другом виде, содержит в себе объекты моделей, ленивое подключение (когда надо)

In [6]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine, future=True, expire_on_commit=False) #future - использование второй версии, expire_on_commit - убиваем на commitу

session = Session() #объект сессии
session_2 = Session()

#### Select

In [7]:
from sqlalchemy import select

query = select(Tracks).where(Tracks.Name.like('%name%'))

retr_1 = session.execute(query).scalar() #scalar - преобразование результата в объект
retr_2 = session.execute(query).scalar()

retr_3 = session.execute(select(Tracks).where(Tracks.Name.like('%name%'))).scalar()

res = retr_1 is retr_2, retr_1 is retr_3, retr_2 is retr_3

res

2024-10-13 13:30:09,559 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-10-13 13:30:09,567 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks 
WHERE tracks."Name" LIKE ?


INFO:sqlalchemy.engine.Engine:SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks 
WHERE tracks."Name" LIKE ?


2024-10-13 13:30:09,570 INFO sqlalchemy.engine.Engine [generated in 0.00319s] ('%name%',)


...

2024-10-13 13:30:09,584 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks 
WHERE tracks."Name" LIKE ?


INFO:sqlalchemy.engine.Engine:SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks 
WHERE tracks."Name" LIKE ?


2024-10-13 13:30:09,587 INFO sqlalchemy.engine.Engine [cached since 0.02089s ago] ('%name%',)


INFO:sqlalchemy.engine.Engine:[cached since 0.02089s ago] ('%name%',)


(True, True, True)

Вообще запросы такие же, как и в Query Builder

#### Insert

In [ ]:
new_track = Tracks(track_id=100000, MediaTypeId=1, Name='new_track', GenreId=1, UnitPrice=1.0, Milliseconds=15)
session.add(new_track)
# ничего не сделал...

In [ ]:
res = session.execute(select(Tracks).filter_by(Name='new_track')).scalar() #а вот тут случился insert
res is new_track

2022-12-04 15:08:41,458 INFO sqlalchemy.engine.Engine INSERT INTO tracks ("TrackId", "Name", "GenreId", "UnitPrice", "MediaTypeId", "Milliseconds") VALUES (?, ?, ?, ?, ?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO tracks ("TrackId", "Name", "GenreId", "UnitPrice", "MediaTypeId", "Milliseconds") VALUES (?, ?, ?, ?, ?, ?)


2022-12-04 15:08:41,464 INFO sqlalchemy.engine.Engine [generated in 0.00659s] (100000, 'new_track', 1, 1.0, 1, 15)


INFO:sqlalchemy.engine.Engine:[generated in 0.00659s] (100000, 'new_track', 1, 1.0, 1, 15)


2022-12-04 15:08:41,470 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks 
WHERE tracks."Name" = ?


INFO:sqlalchemy.engine.Engine:SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks 
WHERE tracks."Name" = ?


2022-12-04 15:08:41,474 INFO sqlalchemy.engine.Engine [generated in 0.00414s] ('new_track',)


INFO:sqlalchemy.engine.Engine:[generated in 0.00414s] ('new_track',)


True

#### Relations (или же Joins)

In [8]:
from sqlalchemy import ForeignKey, select
from sqlalchemy.orm import relationship

class RelatedTrack(Base):
    __tablename__ = 'tracks'
    __table_args__ = {'extend_existing': True} #дабы не было конфликтов (то есть ссылка есть всегда)


    track_id = Column(Integer, name='TrackId', primary_key=True) # обязательно должен быть PrimaryKey
    Name = Column(String)
    GenreId = Column(Integer, ForeignKey('genres.GenreId'))
    UnitPrice = Column(Float)
    MediaTypeId = Column(Integer)
    Milliseconds = Column(Integer)

    genre = relationship('Genre', back_populates='tracks')

class Genre(Base):
    __tablename__ = 'genres'
    __table_args__ = {'extend_existing': True}

    GenreId = Column(Integer, primary_key=True)
    Name = Column(String)

    tracks = relationship("RelatedTrack", back_populates='genre', uselist=True) #uselist - список

In [9]:
genre_1 = session.execute(select(Genre).filter_by(GenreId=1)).scalar()
len(genre_1.tracks)

2024-10-13 13:30:57,531 INFO sqlalchemy.engine.Engine SELECT genres."GenreId", genres."Name" 
FROM genres 
WHERE genres."GenreId" = ?


INFO:sqlalchemy.engine.Engine:SELECT genres."GenreId", genres."Name" 
FROM genres 
WHERE genres."GenreId" = ?


2024-10-13 13:30:57,535 INFO sqlalchemy.engine.Engine [generated in 0.00401s] (1,)


INFO:sqlalchemy.engine.Engine:[generated in 0.00401s] (1,)


2024-10-13 13:30:57,542 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId" AS "tracks_TrackId", tracks."Name" AS "tracks_Name", tracks."GenreId" AS "tracks_GenreId", tracks."UnitPrice" AS "tracks_UnitPrice", tracks."MediaTypeId" AS "tracks_MediaTypeId", tracks."Milliseconds" AS "tracks_Milliseconds" 
FROM tracks 
WHERE ? = tracks."GenreId"


INFO:sqlalchemy.engine.Engine:SELECT tracks."TrackId" AS "tracks_TrackId", tracks."Name" AS "tracks_Name", tracks."GenreId" AS "tracks_GenreId", tracks."UnitPrice" AS "tracks_UnitPrice", tracks."MediaTypeId" AS "tracks_MediaTypeId", tracks."Milliseconds" AS "tracks_Milliseconds" 
FROM tracks 
WHERE ? = tracks."GenreId"


2024-10-13 13:30:57,545 INFO sqlalchemy.engine.Engine [generated in 0.00306s] (1,)


INFO:sqlalchemy.engine.Engine:[generated in 0.00306s] (1,)


1297

In [10]:
genre_1.tracks[0].Name

'For Those About To Rock (We Salute You)'

In [11]:
genre_1.tracks[0].genre is genre_1 #есть обратная связь

True

#### Фильтрация по связям

In [12]:
tracks = session.execute(select(RelatedTrack).join(Genre).where(Genre.Name.like('%Rock%')).limit(5)).scalars().all()
for track in tracks:
    print(track.genre.Name)

2024-10-13 13:31:12,009 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks JOIN genres ON genres."GenreId" = tracks."GenreId" 
WHERE genres."Name" LIKE ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.Engine:SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks JOIN genres ON genres."GenreId" = tracks."GenreId" 
WHERE genres."Name" LIKE ?
 LIMIT ? OFFSET ?


2024-10-13 13:31:12,012 INFO sqlalchemy.engine.Engine [generated in 0.00339s] ('%Rock%', 5, 0)


INFO:sqlalchemy.engine.Engine:[generated in 0.00339s] ('%Rock%', 5, 0)


Rock
Rock
Rock
Rock
Rock


## Животное дня

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Wied%27s_Marmoset_at_Blank_Park_Zoo.gk.jpg/1920px-Wied%27s_Marmoset_at_Blank_Park_Zoo.gk.jpg)

Это игрунка Жоффруа. Меня всегда поражало, что морда очень похожа на человеческое лицо. А еще у нее есть симпатричный вид (то есть животное, которое живет в одном ареале) - это Золотистоголовая львиная игрунка

![](https://upload.wikimedia.org/wikipedia/commons/c/c8/Tamarin_Tête_de_lion.jpg)

Они часто путешествуют вместе с группами игрунок Куля, ищет с ними пищу и предупреждает о нападении хищников. Социальное поведение у игрунок Куля сильно развито, большую часть дня животные проводят в совместных поисках пищи, взаимной чистке и играх. О молодняке проявляют заботу все члены группы. Повзрослев, самки остаются в родной группе, а самцы покидают её. (у них тоже матриархат)